# Georgia Tech Summer 2025 MSA Practicum Project

### 2. Data Retrieval

**Hosted on [Hypertrial.ai](https://www.hypertrial.ai/university-projects/georgia-tech)**

---

In [4]:
import pandas as pd
import requests
from io import StringIO

## 📊 Bitcoin Historical Data (Coin Metrics)

As part of the Stacking Sats project, you’ll be working with a daily historical dataset of Bitcoin metrics provided by [Coin Metrics](https://coinmetrics.io/). This dataset is widely used in industry and research to understand Bitcoin’s market dynamics, and it's updated daily through a public GitHub repository.

You’ll use this data to develop, backtest, and evaluate Bitcoin accumulation strategies over multiple years. 

### 📁 Dataset Information
- **Source**: Coin Metrics Community Data
- **URL**: [btc.csv](https://raw.githubusercontent.com/coinmetrics/data/master/csv/btc.csv)
- **License**: Creative Commons Attribution-NonCommercial 4.0 International (CC BY-NC 4.0)
- **Update Frequency**: Daily

This CSV file contains a wide range of metrics. You're encouraged to explore them all to enhance your strategy design.

| Field                                               | Description                                                                                      | Units / Notes         |
|-----------------------------------------------------|--------------------------------------------------------------------------------------------------|------------------------|
| time                                                | End-of-day date (YYYY-MM-DD, 00:00 UTC inclusive)                                                | date                   |
| AdrActCnt                                           | Unique addresses that sent or received value that day                                            | addresses              |
| AdrBal1in100KCnt … AdrBal1in10BCnt                  | Address count whose balance ≥ current supply ÷ denominator (e.g. AdrBal1in100KCnt = ≥ 0.001 % supply) | addresses              |
| AdrBalCnt                                           | Addresses with non-zero balance                                                                  | addresses              |
| AdrBalNtv0.001Cnt … AdrBalNtv100KCnt                | Address count with balance ≥ X BTC (native units)                                                | addresses              |
| AdrBalUSD1Cnt … AdrBalUSD10MCnt                     | Address count with BTC balance worth ≥ X USD at EOD price                                        | addresses              |
| AssetCompletionTime / AssetEODCompletionTime        | Timestamps when CM finished processing intraday & EOD files; useful for data QA                  | datetime               |
| BlkCnt                                              | Blocks mined                                                                                     | blocks                 |
| BlkSizeMeanByte / BlkWghtMean / BlkWghtTot          | Mean block size (bytes), mean block weight, total block weight                                   | bytes / weight         |
| CapAct1yrUSD                                        | Realised cap of coins moved ≤ 1 year ago (“active 1 yr”)                                         | USD                    |
| CapMVRVCur / CapMVRVFF                              | MVRV ratios using current or free-float supply                                                   | dimensionless          |
| CapMrktCurUSD / CapMrktEstUSD / CapMrktFFUSD        | Current, estimated and free-float market cap                                                     | USD                    |
| CapRealUSD                                          | Realised capitalisation (sum of UTXO cost basis)                                                 | USD                    |
| DiffLast / DiffMean                                 | Difficulty of last block and mean difficulty that day                                            | dimensionless          |
| FeeByteMeanNtv                                      | Mean fee per byte                                                                                | BTC/byte               |
| FeeMeanNtv / FeeMeanUSD / FeeMedNtv / FeeMedUSD     | Mean & median tx fee                                                                             | BTC / USD              |
| FeeTotNtv / FeeTotUSD                               | Aggregate fees paid in blockspace                                                                | BTC / USD              |
| FlowInExNtv / FlowInExUSD                           | Native/fiat value flowing into exchange-tagged addresses                                         | BTC / USD              |
| FlowOutExNtv / FlowOutExUSD                         | Value flowing out of exchanges                                                                   | BTC / USD              |
| FlowTfrFromExCnt                                    | Count of transfers initiated by exchange addresses                                               | transfers              |
| HashRate / HashRate30d                              | Estimated PoW hash-rate (daily, 30-day MA)                                                       | TH/s                   |
| IssContNtv / IssContUSD                             | Newly-minted BTC (block subsidy) and its USD value                                               | BTC / USD              |
| IssContPctDay / IssContPctAnn                       | Daily & annualised on-chain inflation rates                                                      | %                      |
| IssTotNtv / IssTotUSD                               | Cumulative issued supply (native / USD)                                                          | BTC / USD              |
| NDF                                                 | Network Distribution Factor — share of supply held by addresses ≥ 0.01 % of supply               | dimensionless          |
| NVTAdj / NVTAdj90 / NVTAdjFF / NVTAdjFF90           | Network-value-to-(adjusted) transfer-value ratios (single-day & 90-day, total/free-float)        | dimensionless          |
| PriceBTC / PriceUSD                                 | BTC price in BTC (=1) and USD (Coin Metrics reference)                                           | BTC / USD              |
| ROI1yr / ROI30d                                     | 1-year & 30-day unlevered return on investment                                                   | %                      |
| ReferenceRate, ReferenceRateETH, EUR, USD           | CM hourly reference rates (snapshotted EOD) vs BTC, ETH, EUR, USD                                | unit varies            |
| RevAllTimeUSD                                       | Aggregate miner revenue since genesis                                                            | USD                    |
| RevHashNtv / RevHashRateNtv / RevHashRateUSD / USD  | Miner revenue per hash or per TH                                                                 | BTC / BTC/TH / USD/TH  |
| RevNtv / RevUSD                                     | Daily total miner revenue (subsidy + fees)                                                       | BTC / USD              |
| SER                                                 | Supply Equality Ratio — poorest (< 0.00001 % supply) ÷ richest 1%                                | dimensionless          |
| SplyAct1d … SplyAct10yr, SplyActEver                | BTC that moved within given look-back window; Ever = cumulative unique supply ever spent         | BTC                    |
| SplyActPct1yr                                       | % of circulating supply active in last 365 days                                                  | %                      |
| SplyCur / SplyExpFut10yr / SplyFF                   | Circulating, projected (10 y) and free-float supply                                              | BTC                    |
| SplyAdrBal…                                         | Same thresholds as AdrBal… but measured in BTC held, not address count                           | BTC / USD              |
| SplyMiner0HopAllNtv / USD, SplyMiner1HopAllNtv / USD| BTC (and USD value) still held by miner wallets (0-hop) or miner + 1-hop wallets                 | BTC / USD              |
| SplyAdrTop100 / Top10Pct / Top1Pct                  | Supply held by top entities (rank or percentile)                                                 | BTC                    |
| TxCnt / TxCntSec                                    | Count of transactions, and average TPS                                                           | tx / tx·s⁻¹            |
| TxTfrCnt                                            | Value-transferring outputs (CM “transfer” heuristic)                                             | transfers              |
| TxTfrValAdjNtv / USD                                | Adjusted transfer value (ex-self-change & known jitter)                                          | BTC / USD              |
| TxTfrValMeanNtv / USD, TxTfrValMedNtv / USD         | Mean & median adjusted transfer sizes                                                            | BTC / USD              |
| VelCur1yr                                           | Coin velocity using trailing-year adjusted transfer volume                                       | 1/yr                   |
| VtyDayRet180d / VtyDayRet30d                        | Realised volatility of daily log-returns (180-d, 30-d)                                           | %                      |
| principal_market_price_usd / principal_market_usd   | Price and notional of CM-selected “principal” BTC market at EOD                                 | USD                    |


In [6]:
# Coin Metrics BTC CSV (raw GitHub URL)
url = "https://raw.githubusercontent.com/coinmetrics/data/master/csv/btc.csv"

# Download the content
response = requests.get(url)
response.raise_for_status()  # raises an error for bad responses

# Parse CSV content
btc_df = pd.read_csv(StringIO(response.text))

# Show the df
btc_df

/var/folders/zn/3r5r5myd1dq34m6jv1g5rq7m0000gn/T/ipykernel_1409/4280389743.py:9: DtypeWarning: Columns (147) have mixed types. Specify dtype option on import or set low_memory=False.
  btc_df = pd.read_csv(StringIO(response.text))


,time,AdrActCnt,AdrBal1in100KCnt,AdrBal1in100MCnt,AdrBal1in10BCnt,AdrBal1in10KCnt,AdrBal1in10MCnt,AdrBal1in1BCnt,AdrBal1in1KCnt,AdrBal1in1MCnt,...,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,VelCur1yr,VtyDayRet180d,VtyDayRet30d,principal_market_price_usd,principal_market_usd
0,2009-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-01-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5980,2025-05-19,718046.0,11283.0,2967667.0,19943914.0,896.0,531661.0,9311741.0,46.0,100222.0,...,1.515012e+10,0.256888,27131.472643,0.001013,106.940251,4.424386,0.025910,0.018900,106476.34,coinbase-btc-usd-spot
5981,2025-05-20,739140.0,11278.0,2967993.0,19942680.0,898.0,531487.0,9310886.0,46.0,100188.0,...,1.334977e+10,0.249574,26677.764378,0.001000,106.893330,4.424584,0.025706,0.018866,105616.18,coinbase-btc-usd-spot
5982,2025-05-21,796779.0,11293.0,2966772.0,19936930.0,901.0,530800.0,9308020.0,45.0,100149.0,...,1.669525e+10,0.325844,35746.642265,0.000716,78.580491,4.422780,0.025775,0.018858,106883.27,coinbase-btc-usd-spot
5983,2025-05-22,868642.0,11319.0,2964524.0,19931264.0,899.0,530360.0,9299911.0,45.0,100112.0,...,1.754588e+10,0.316972,35335.117678,0.000846,94.361949,4.425488,0.025782,0.015156,109693.48,coinbase-btc-usd-spot


Read more about the data spec [here](https://gitbook-docs.coinmetrics.io/network-data/network-data-overview/addresses/address-balances?utm_source=chatgpt.com)